In [8]:
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

In [4]:
class QAPipeline(object):
    def __init__(self):
        self.model = "deepset/minilm-uncased-squad2"

    def create_pipeline(self, index, embedding_model):
        document_store = ElasticsearchDocumentStore(
            host='44.212.62.175',
            port='9200',
            username='',
            password='',
            index=index,
            similarity="cosine",
            embedding_dim=768
        )

        retriever = EmbeddingRetriever(
            document_store=document_store,
            embedding_model=embedding_model,
        )
        # document_store.update_embeddings(retriever)
        reader = FARMReader(model_name_or_path=self.model, use_gpu=True)
        pipe = ExtractiveQAPipeline(reader, retriever)
        self.pipeline = pipe

    def predict(self, query):
        prediction = self.pipeline.run(
            query=query, 
            params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 5}}
        )
        return prediction

### Pretrained retriever

In [15]:
pipeline = QAPipeline()
pipeline.create_pipeline('bioasq-pretrained', "sentence-transformers/msmarco-distilbert-base-tas-b")

In [16]:
prediction = pipeline.predict("Which drug is benserazide usually co-administered with?")

Inferencing Samples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.80 Batches/s]


In [17]:
print_answers(prediction, details="minimum")


Query: Which drug is benserazide usually co-administered with?
Answers:
[   {   'answer': 'benzodiazepines',
        'context': 'verse sedation caused by intoxication with benzodiazepines '
                   'alone or benzodiazepines in combination with other agents, '
                   'but it should not be used when c'},
    {   'answer': 'carbamazepine',
        'context': 'using this agent in patients who have co-ingested chloral '
                   'hydrate to carbamazepine or whose ECG shows abnormalities '
                   'typical to those seen after overdo'},
    {   'answer': 'NNRTI in combination with two nucleoside analogs',
        'context': 'ine antiretroviral therapies typically include one NNRTI '
                   'in combination with two nucleoside analogs. In 2008, the '
                   'next-generation NNRTI etravirine was'},
    {   'answer': 'DOPA decarboxylase inhibitor, abolished the antinociceptive '
                  'effect of L-DOPA',
        'con

### Adapted retriever

In [10]:
pipeline = QAPipeline()
pipeline.create_pipeline('bioasq-retriever-adpated', "sentence-transformers/msmarco-distilbert-base-tas-b")

In [13]:
prediction = pipeline.predict("Which drug is benserazide usually co-administered with?")

Inferencing Samples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.22 Batches/s]


In [14]:
print_answers(prediction, details="minimum")


Query: Which drug is benserazide usually co-administered with?
Answers:
[   {   'answer': 'PDI',
        'context': ' indicating that there was induction of ALAAD by the '
                   'co-administration of PDI. Despite this induction of ALAAD, '
                   'in groups 3 and 4, with half the daily'},
    {   'answer': 'L-DOPA',
        'context': 'tagonists (0.01, 0.1, 1.0 mg/kg each; i.p.), or vehicle 30 '
                   'min prior to L-DOPA (6 mg/kg; i.p.)+Benserazide (15 mg/kg; '
                   'i.p.). SCH23390 (0.1 and 1.0 mg/'},
    {   'answer': 'DOPA decarboxylase inhibitor, abolished the antinociceptive '
                  'effect of L-DOPA',
        'context': 'inistration of benserazide (i.t.), a DOPA decarboxylase '
                   'inhibitor, abolished the antinociceptive effect of L-DOPA. '
                   'The L-DOPA-induced antinociception '},
    {   'answer': 'placebo',
        'context': ', double-blind study randomized patients to receive 